# mbart-large-cc25 finetuning
Example notebook for ja-en finetuning based on [facebook/mbart-large-cc25](https://huggingface.co/facebook/mbart-large-cc25)

## Install pytorch

In [2]:
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu111/torch-1.9.0%2Bcu111-cp36-cp36m-linux_x86_64.whl (2041.3 MB)
  Using cached https://download.pytorch.org/whl/cu111/torchvision-0.10.0%2Bcu111-cp36-cp36m-linux_x86_64.whl (23.2 MB)
  Using cached torchaudio-0.9.0-cp36-cp36m-manylinux1_x86_64.whl (1.9 MB)
  Using cached typing_extensions-3.10.0.2-py3-none-any.whl (26 kB)
  Using cached dataclasses-0.8-py3-none-any.whl (19 kB)
  Using cached numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl (14.8 MB)
  Using cached Pillow-8.3.2-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


## Install dependencies

In [8]:
!pip install --upgrade pip
!pip install transformers["ja"] numpy pandas sentencepiece fairseq
!pip install -U jupyter ipywidgets 

     |████████████████████████████████| 1.7 MB 9.5 MB/s            
     |████████████████████████████████| 145 kB 109.3 MB/s            
     |████████████████████████████████| 90 kB 1.8 MB/s              
     |████████████████████████████████| 2.0 MB 70.3 MB/s            
     |████████████████████████████████| 112 kB 101.9 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 74 kB 438 kB/s             
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=a8ad19a8f0e82cb59bd7c859a476aeee913e4c99dd9e363774efea8a13a21a97
  Stored in directory: /root/.cache/pip/wheels/a8/04/35/9449686f1c26ff16f6224dc942e108329f3782185802ec6b93
Successfully built antlr4-python3-runtime


## Download training dataset
今回はJESCのデータセットを利用します。  
このような大規模なデータセットを公開していただいていることに感謝します。  
This time we will use the [JESC dataset](https://nlp.stanford.edu/projects/jesc/index_ja.html) .  
Thank you for publishing such a large dataset.

In [5]:
!wget "https://nlp.stanford.edu/projects/jesc/data/split.tar.gz"
!tar -zxvf split.tar.gz
!ls split

dev  test  train


## Create training data for tokenizer
Sentencepieceの学習に利用するデータを作成します。  
Create the data used for learning the sentence piece.

In [8]:
res = []
for line in open('split/train', 'r', encoding='utf-8'):
    text = line.split('\t')
    text = [t.rstrip('\n') for t in text]
    res.extend(text)
for line in open('split/dev', 'r', encoding='utf-8'):
    text = line.split('\t')
    text = [t.rstrip('\n') for t in text]
    res.extend(text)
for line in open('split/test', 'r', encoding='utf-8'):
    text = line.split('\t')
    text = [t.rstrip('\n') for t in text]
    res.extend(text)

print(len(res))
with open('tmp.txt', 'w') as f:
    for d in res:
        f.write("%s\n" % d)

5602776


In [3]:
!tail tmp.txt

am i free to go?
行っていいのですか?
you'll definitely become a good nurse.
あんた きっと いい看護師になるよ。
and isn't that what your facemash was about?
フェイスマッシュも同じだ
what is it this time?
ロケットパンチ止めようとしてんぞ! どういう状況だ? これ!
you poured your father's remains in and closed it.
一度 封をはがして灰を入れ また封をしても―


## Training tokenizer
Sentencepieceの学習をします。  

In [5]:
import sentencepiece as spm

# @NOTE
# ボキャブラリーのサイズは適宜変更してください。
# Please change the size of the vocabulary accordingly.
spm.SentencePieceTrainer.Train("--input=tmp.txt --model_prefix=new_spm_model --vocab_size=64000 --vocabulary_output_piece_score=false --model_type=bpe")

aaa


## Download pre-trained model
後述の作業で必要になるので、huggingfaceではなくfairseqから直接事前学習済みモデルをダウンロードしてきます  
Download the pre-trained model directly from fairseq instead of huggingface as you will need it for the tasks described below.

In [21]:
!wget "https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.cc25.v2.tar.gz"
!tar -zxvf mbart.cc25.v2.tar.gz
!ls mbart.cc25.v2

--2021-10-13 12:16:27--  https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.cc25.v2.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5618826847 (5.2G) [application/gzip]
Saving to: ‘mbart.cc25.v2.tar.gz’

mbart.cc25.v2.tar.g 100%[===================>]   5.23G  52.5MB/s    in 2m 1s   

2021-10-13 12:18:39 (44.4 MB/s) - ‘mbart.cc25.v2.tar.gz’ saved [5618826847/5618826847]

mbart.cc25.v2/
mbart.cc25.v2/sentence.bpe.model
tar: mbart.cc25.v2/sentence.bpe.model: Cannot change ownership to uid 1185301073, gid 1185301073: Invalid argument
mbart.cc25.v2/dict.txt
tar: mbart.cc25.v2/dict.txt: Cannot change ownership to uid 1185301073, gid 1185301073: Invalid argument
mbart.cc25.v2/model.pt
tar: mbart.cc25.v2/model.pt: Cannot change ownership to uid 1185301073, gid 1185301073: In

## Weight reduction of pre trained model
ベースとなるモデル([facebook/mbart-large-cc25](https://huggingface.co/facebook/mbart-large-cc25))の軽量化をしていきます。  
この工程は主にベースモデルのサイズが巨大でバッチサイズが1でも学習ができないという問題を解決するために実行します。  
※[こちらのissue](https://github.com/pytorch/fairseq/issues/2120)で語られている問題です。  
We will reduce the weight of the base model.  
This process is mainly performed to solve the problem that the size of the base model is huge and even if the batch size is 1, it cannot be trained.  
See this [issue](https://github.com/pytorch/fairseq/issues/2120) for more details.

### 補足
ここで実行する解決方法は、先ほど作成したボキャブラリーファイルをベースに、必要な単語の情報を残し不要な単語の情報を削除していくものです。  
ベースとなっているモデルではおよそ25万の単語が収録されたボキャブラリーを使用していますが、ここには25種類の言語の単語が収録されています。  
ファインチューニングでは目的となる言語を絞り込める(今回で言えば日本語と英語だけ)と思うので、必要な単語以外は切り落としてしまおうという手法です。  
[こちらで提案されている](https://github.com/pytorch/fairseq/issues/2120#issuecomment-633460071)ものです。  
以下のコードは主に[こちらのコメントのコード](https://github.com/pytorch/fairseq/issues/2120#issuecomment-647429120)がベースになっています。  
素敵な手法を提案された[fansiawang氏](https://github.com/fansiawang)とサンプルコードをコメントしてくださった[ddaspit氏](https://github.com/ddaspit)に感謝申し上げます。

### Formatting vocab
先ほどつくったvocabファイルはそのままではこのあとの工程で使えないので加工します。  
The vocab file created earlier cannot be used as it is in the subsequent process, so it will be processed.

In [17]:
edited = []
for line in open("new_spm_model.vocab", 'r', encoding='utf-8'):
    if line in ["<unk>\n", "<s>\n", "</s>\n"]:
        continue
    new_line = line.rstrip('\n') + " 1\n"
    edited.append(new_line)

with open('new_dict.txt', 'w') as f:
    for e in edited:
        f.write(e)

In [18]:
!ls

Untitled.ipynb	new_spm_model.model  split	   tmp.txt
new_dict.txt	new_spm_model.vocab  split.tar.gz  translation.ipynb


### Reduce to create a new model.
軽量化して新しいモデルを作成します。  

In [29]:
!mkdir reduced_model
!ls

Untitled.ipynb	      new_dict.txt	   reduced_model  tmp.txt
mbart.cc25.v2	      new_spm_model.model  split	  translation.ipynb
mbart.cc25.v2.tar.gz  new_spm_model.vocab  split.tar.gz


In [31]:
from fairseq.data import Dictionary
from transformers import (
    MBartForConditionalGeneration, MBartTokenizer, MBartConfig
)
from typing import List
import torch

In [32]:
langs = [
    "ar_AR",
    "cs_CZ",
    "de_DE",
    "en_XX",
    "es_XX",
    "et_EE",
    "fi_FI",
    "fr_XX",
    "gu_IN",
    "hi_IN",
    "it_IT",
    "ja_XX",
    "kk_KZ",
    "ko_KR",
    "lt_LT",
    "lv_LV",
    "my_MM",
    "ne_NP",
    "nl_XX",
    "ro_RO",
    "ru_RU",
    "si_LK",
    "tr_TR",
    "vi_VN",
    "zh_CN"
]

def load_dict(langs: List[str], path: str) -> Dictionary:
    d = Dictionary.load(path)
    for ll in langs:
        d.add_symbol(f"[{ll}]")
    d.add_symbol("<mask>")
    d.add_symbol("<pad>")
    return d


pre_dict = load_dict(langs, "./mbart.cc25.v2/dict.txt")
ft_dict = load_dict(langs, "./new_dict.txt")

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-cc25")
org_sd = model.state_dict()
resized_sd = model.state_dict()

mapping: List[int] = []
for i in range(len(ft_dict)):
    word = ft_dict[i]
    mapping.append(pre_dict.index(word))

for name in ["model.encoder.embed_tokens.weight", "model.decoder.embed_tokens.weight", "model.shared.weight", "lm_head.weight"]:
    pre_tensor: torch.Tensor = org_sd[name]
    ft_tensor = torch.zeros(
        [len(ft_dict), 1024], dtype=pre_tensor.dtype, layout=pre_tensor.layout, device=pre_tensor.device,
    )
    for ft_i, pre_i in enumerate(mapping):
        ft_tensor[ft_i] = pre_tensor[pre_i]
    resized_sd[name] = ft_tensor
resized_sd["final_logits_bias"] = resized_sd["final_logits_bias"][:, :len(ft_dict)]

config = MBartConfig.from_pretrained("facebook/mbart-large-cc25")
config.vocab_size = len(ft_dict)
print(config)
new_model = MBartForConditionalGeneration.from_pretrained(None, config=config, state_dict=resized_sd)
new_model.save_pretrained("./reduced_model")

MBartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_length": 1024,
  "max_position_embeddings": 1024,
  "model_type": "mbart",
  "normalize_before": true,
  "normalize_embedding": true,
  "num_beams": 5,
  "nu

In [33]:
!ls reduced_model

config.json  pytorch_model.bin


以上でベースモデルの軽量化が完了します。  
ここからは `reduced_model` ディレクトリをpre-trainedモデルとして利用していきます。  
This completes the weight reduction of the base model.  
From now on, we will use the `reduced_model` directory as a pre-trained model.

## Preparation of Tokenizer
今のままでは不足しているファイルがあるので取得します  
Get the missing files

In [35]:
tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-cc25")
tokenizer.save_pretrained("./reduced_model")

('./reduced_model/tokenizer_config.json',
 './reduced_model/special_tokens_map.json',
 './reduced_model/sentencepiece.bpe.model',
 './reduced_model/added_tokens.json')

モデルファイルは先ほど作成したもので上書きします  
Overwrite the model file with the one you created earlier

In [36]:
!mv ./new_spm_model.model ./reduced_model/sentencepiece.bpe.model

In [38]:
!ls -al ./reduced_model

total 1643992
drwxr-xr-x 3 root root       6144 Oct 13 12:49 .
drwxr-xr-x 6 root root       6144 Oct 13 12:49 ..
drwxr-xr-x 2 root root       6144 Oct 13 12:47 .ipynb_checkpoints
-rw-r--r-- 1 root root       1364 Oct 13 12:36 config.json
-rw-r--r-- 1 root root 1681991993 Oct 13 12:36 pytorch_model.bin
-rw-r--r-- 1 root root    1427810 Oct 13 11:31 sentencepiece.bpe.model
-rw-r--r-- 1 root root        494 Oct 13 12:46 special_tokens_map.json
-rw-r--r-- 1 root root        686 Oct 13 12:46 tokenizer_config.json


以上でモデルとトークナイザー両方が `reduced_model` ディレクトリから呼べるようになります  
Now both the model and the tokenizer can be called from the `reduced_model` directory.

In [39]:
model = MBartForConditionalGeneration.from_pretrained("./reduced_model")
tokenizer = MBartTokenizer.from_pretrained("./reduced_model")

## Training
モデルとトークナイザーの準備ができたので、トレーニングを実行します。  
トレーニングのコードは[こちら](https://www.kaggle.com/ajax0564/mbart-finetuning-hintoenglish-translation)を参考にしています  
Now that the model and tokenizer are ready, it's time to start training.  
[Here](https://www.kaggle.com/ajax0564/mbart-finetuning-hintoenglish-translation) is the reference code.

In [43]:
!mkdir output

In [44]:
from transformers import (
    Seq2SeqTrainingArguments, Seq2SeqTrainer
)
import numpy as np
import re

result_dir = "./output"

In [45]:
def data_collator(features: list):
    x = [f["translation"]["ja"] for f in features]
    y = [f["translation"]["en"] for f in features]
    inputs = tokenizer(x, return_tensors="pt", padding='max_length', truncation=True, max_length=32)
    with tokenizer.as_target_tokenizer():
        inputs['labels'] = tokenizer(y, return_tensors="pt", padding='max_length', truncation=True, max_length=48)['input_ids']
    return inputs

tokenizer = MBartTokenizer.from_pretrained("./reduced_model", src_lang="ja_XX", tgt_lang="en_XX")
tokenizer.save_pretrained(result_dir)

('./output/tokenizer_config.json',
 './output/special_tokens_map.json',
 './output/sentencepiece.bpe.model',
 './output/added_tokens.json')

In [48]:
train_data = []
eval_data = []

for line in open("./split/train", "r", encoding='utf-8'):
    text = line.split('\t')
    train_data.append(
        {"translation": {
            "ja": text[1].rstrip('\n'),
            "en": text[0].rstrip('\n')
        }}
    )
print(f"train_data size: {len(train_data)}")

for line in open("./split/dev", "r", encoding='utf-8'):
    text = line.split('\t')
    eval_data.append(
        {"translation": {
            "ja": text[1].rstrip('\n'),
            "en": text[0].rstrip('\n')
        }}
    )
print(f"eval_data size: {len(eval_data)}")

train_data size: 2000
eval_data size: 2000


In [49]:
# Hyperparameters
batch_size = 1
learning_rate = 3e-5
epochs = 1


In [50]:
model = MBartForConditionalGeneration.from_pretrained("./reduced_model")

args = Seq2SeqTrainingArguments(output_dir=result_dir,
                                do_train=True,
                                do_eval=True,
                                per_device_train_batch_size=batch_size,
                                per_device_eval_batch_size=batch_size,
                                learning_rate=learning_rate,
                                num_train_epochs=epochs,
                                evaluation_strategy="epoch",
                                )

trainer = Seq2SeqTrainer(model=model,
                         args=args,
                         data_collator=data_collator,
                         train_dataset=train_data,
                         eval_dataset=eval_data,
                         )

In [51]:
trainer.train()
trainer.save_model(result_dir)

***** Running training *****
  Num examples = 2000
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 2000


Epoch,Training Loss,Validation Loss
1,1.045500,0.961348


Saving model checkpoint to ./output/checkpoint-500
Configuration saved in ./output/checkpoint-500/config.json
Model weights saved in ./output/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./output/checkpoint-1000
Configuration saved in ./output/checkpoint-1000/config.json
Model weights saved in ./output/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./output/checkpoint-1500
Configuration saved in ./output/checkpoint-1500/config.json
Model weights saved in ./output/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./output/checkpoint-2000
Configuration saved in ./output/checkpoint-2000/config.json
Model weights saved in ./output/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 1


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./output
Configuration saved in ./output/config.json
Model weights saved in ./output/pytorch_model.bin


## Inference
できあがったモデルを使って推論を実行してみます  
Let's perform inference using the resulting model.

In [52]:
model = MBartForConditionalGeneration.from_pretrained("./output")
tokenizer = MBartTokenizer.from_pretrained("./output")

loading configuration file ./output/config.json
Model config MBartConfig {
  "_name_or_path": "./reduced_model",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_length": 1024,
  "max_position_embeddings": 1024,
  "model

In [53]:
sentence = "おはよう"
inputs = tokenizer(sentence, return_tensors="pt")
translated_tokens = model.generate(**inputs, decoder_start_token_id=tokenizer.lang_code_to_id["en_XX"], early_stopping=True, max_length=48)
pred = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
print(f"日本語 - {sentence}: English - {pred}")

日本語 - おはよう: English - it's okay. it's okay.
